In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy.stats import boxcox

# suppress warnings about "value is trying to be set on a copy of a slice from a DataFrame"
#pd.options.mode.chained_assignment = None  # default='warn'

In [7]:
life_df = pd.read_csv("Data/life_expectancy.csv")
life_df

,country,country_code,region,year,life_expect,life_exp60,adult_mortality,infant_mort,age1-4mort,alcohol,...,che_gdp,une_pop,une_infant,une_life,une_hiv,une_gni,une_poverty,une_edu_spend,une_literacy,une_school
0,Angola,AGO,Africa,2000,47.33730,14.73400,383.5583,0.137985,0.025695,1.47439,...,1.90860,16395.473,122.2,46.522,1.0,2530.0,32.3,2.60753,NaN,NaN
1,Angola,AGO,Africa,2001,48.19789,14.95963,372.3876,0.133675,0.024500,1.94025,...,4.48352,16945.753,118.9,47.059,1.1,2630.0,NaN,NaN,67.40542,NaN
2,Angola,AGO,Africa,2002,49.42569,15.20010,354.5147,0.128320,0.023260,2.07512,...,3.32946,17519.417,115.1,47.702,1.2,3180.0,NaN,NaN,NaN,NaN
3,Angola,AGO,Africa,2003,50.50266,15.39144,343.2169,0.122040,0.021925,2.20275,...,3.54797,18121.479,110.8,48.440,1.3,3260.0,NaN,NaN,NaN,NaN
4,Angola,AGO,Africa,2004,51.52863,15.56860,333.8711,0.115700,0.020545,2.41274,...,3.96720,18758.145,106.2,49.263,1.3,3560.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3106,Lao People's Democratic Republic,LAO,Western Pacific,2012,64.49448,16.57977,209.2666,0.055840,0.004845,6.29833,...,2.07638,6444.530,47.5,65.267,0.3,4570.0,22.7,1.82147,NaN,NaN
3107,Lao People's Democratic Republic,LAO,Western Pacific,2013,64.79716,16.64219,205.1459,0.055350,0.004600,6.42186,...,2.39985,6541.304,45.6,65.717,0.3,4980.0,NaN,3.23381,NaN,NaN
3108,Lao People's Democratic Republic,LAO,Western Pacific,2014,65.10757,16.70787,200.8204,0.054585,0.004405,6.64447,...,2.29846,6639.756,43.7,66.144,0.3,5440.0,NaN,2.93781,NaN,NaN
3109,Lao People's Democratic Republic,LAO,Western Pacific,2015,65.45008,16.77074,196.6613,0.053465,0.004160,6.83826,...,2.45366,6741.164,42.0,66.546,0.3,5810.0,NaN,NaN,84.66104,NaN


In [8]:
life_df.isna().sum()

country               0
country_code          0
region                0
year                  0
life_expect           0
life_exp60            0
adult_mortality       0
infant_mort           0
age1-4mort            0
alcohol              50
bmi                  34
age5-19thinness      34
age5-19obesity       34
hepatitis           569
measles              19
polio                19
diphtheria           19
basic_water          32
doctors            1331
hospitals          2981
gni_capita          682
gghe-d              100
che_gdp             117
une_pop              37
une_infant            0
une_life              0
une_hiv             741
une_gni             117
une_poverty        2198
une_edu_spend      1286
une_literacy       2540
une_school         2306
dtype: int64

## Functions

Defining functions which will be used later to build a data cleaning pipeline. 

In [9]:
def headers_to_lowercase (df):
    df.columns = df.columns.str.lower()
    return df


def text_to_lowercase (df):
    df = df.applymap(lambda s: s.lower() if type(s) == str else s)
    return df   


def drop_columns (df, list_of_columns_to_drop):    
    df.drop(columns = list_of_columns_to_drop, inplace=True)
    return df


def drop_rows (df, country_drop_list):
    df = df[~df['country'].isin(country_drop_list)]
    return df


def fill_na_in_population_with_values_from_dict (df):
    
    singapore_pop_dict = {2000:4028.871 , 2001:4077.145 , 2002:4104.419 , 2003:4129.225 , 2004: 4177.213 , 2005: 4265.687 , 2006:4402.331 , 2007:4578.630 , 2008:4775.804 , 2009:4966.614 , 2010:5131.172 , 2011:5263.636, 2012:5369.476, 2013:5453.737, 2014:5525.626, 2015:5592.152, 2016:5653.634}
    ukraine_pop_dict = {2014:45111.532, 2015:44921.639, 2016: 44713.702}
    brazil_pop_dict = {2000:174790.340, 2006: 188167.356, 2010:195713.635}
    
    df['une_pop'][df['country']=="singapore"] = df['une_pop'][df['country']=="singapore"].fillna(df['year'].map(singapore_pop_dict))
    df['une_pop'][df['country']=="brazil"] = df['une_pop'][df['country']=="brazil"].fillna(df['year'].map(brazil_pop_dict))
    df['une_pop'][df['country']=="ukraine"] = df['une_pop'][df['country']=="ukraine"].fillna(df['year'].map(ukraine_pop_dict))
    
    return df


def change_col_names (df, column_name_dict):
    df.rename(columns=column_name_dict, inplace= True)
    return df


def replace_nans_with_country_mean (df, list_of_categories):
    for i in list_of_categories:
        df[i] = df[i].fillna(df.groupby('country')[i].transform('mean'))
    return df


def round_floats (df):
    df = df.applymap(lambda s: np.round(s,5) if type(s) == float else s)
    return df 

## Variables for data cleaning

In [14]:
list_of_columns_to_drop = ['country_code', 'hospitals','gni_capita','une_hiv', 'une_poverty', 'une_edu_spend', 'une_literacy','une_school', 'une_infant', 'une_life','hepatitis','life_exp60','infant_mort','bmi','adult_mortality']
country_drop_list = ['somalia','sudan','montenegro','south sudan',"democratic people's republic of korea", 'albania', 'bosnia and herzegovina','syrian arab republic','cuba','djibouti']
column_name_dict = {'life_expect':'life_expectancy','life_exp60':'life_expectancy_at_60','infant_mort':'infant_mortality','age1-4mort':'age_1-4_mortality', 'age5-19thinness':'age_5-19_thinness', 'age5-19obesity':'age_5-19_obesity','polio':'polio_vacc', 'basic_water':'basic_water_access', 'doctors':'doctors_per_10000', 'une_pop':'population', 'une_gni':'gni','diphtheria':'diphteria_vacc','measles':'measles_vacc'}
list_of_categories = ['alcohol','doctors_per_10000','gni','polio_vacc','diphteria_vacc','measles_vacc','basic_water_access','gghe-d','che_gdp']

## Data cleaning pipeline

In [15]:
life_df = (life_df
.pipe(headers_to_lowercase)
.pipe(text_to_lowercase)
.pipe(drop_columns, list_of_columns_to_drop)
.pipe(drop_rows, country_drop_list)
.pipe(fill_na_in_population_with_values_from_dict)
.pipe(change_col_names, column_name_dict)
.pipe(replace_nans_with_country_mean, list_of_categories)
.pipe(round_floats)
)

/var/folders/qw/spw5ptl93b53bn0c_4sqb01m0000gq/T/ipykernel_19101/3291339912.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['une_pop'][df['country']=="brazil"] = df['une_pop'][df['country']=="brazil"].fillna(df['year'].map(brazil_pop_dict))
/var/folders/qw/spw5ptl93b53bn0c_4sqb01m0000gq/T/ipykernel_19101/3291339912.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['une_pop'][df['country']=="ukraine"] = df['une_pop'][df['country']=="ukraine"].fillna(df['year'].map(ukraine_pop_dict))


In [17]:
life_df.isna().sum()

country               0
region                0
year                  0
life_expectancy       0
age_1-4_mortality     0
alcohol               0
age_5-19_thinness     0
age_5-19_obesity      0
measles_vacc          0
polio_vacc            0
diphteria_vacc        0
basic_water_access    0
doctors_per_10000     0
gghe-d                0
che_gdp               0
population            0
gni                   0
dtype: int64

In [18]:
life_df.to_csv("Data/cleaned_life_expectancy_df_visualization.csv", index=False)